First lab Computational intelligence: Set covering

This code was written in collaboration with Lorenzo Bonannella (https://github.com/lorenzobn/computational_intelligence)

In [1]:
import numpy as np
from random import random
from functools import reduce
from queue import PriorityQueue

In [2]:
#costants for the complexity of the problem
PROBLEM_SIZE = 5
NUM_SETS = 10 #SETS == TILES to choose from
SETS = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
#the set of all sets/tiles we can choose from

In [46]:
#to check if we finished
def goal_check(state):
    #whate happens in case of no index? Add initial value
    return np.all(reduce(np.logical_or, [SETS[i] for i in state[0]], np.array([False for _ in range(PROBLEM_SIZE)])))


assert goal_check((set(range(NUM_SETS)), set())), "Problem not solvable" #sanity check => if i select all sets i must have a solution
#otherwise the problem is not solvable


In [47]:
#the quality of a solution depends on the number of selected tiles
#so for each state we compute the cost as the number of selected tiles
def Cost(state):
    return len(state[0])


#here the distance code seen at lesson
def Distance(state):
    already_covered = reduce(
        np.logical_or,
        [SETS[i] for i in state[0]],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )
    if np.all(already_covered):
        return 0
    
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

In [ ]:
#Path search implementation
frontier = PriorityQueue()
initial_state = (set(), set(range(NUM_SETS))) #initial state => no selected sets, all available index

#for starting we add in the frontier the initial state
frontier.put((0, initial_state))

(_, state) = frontier.get()

counter = 0
while not goal_check(state):
    counter += 1
    print(f"{counter}: selected state: {state} with cost {Cost(state)} and distance {Distance(state)}")

    for action in state[1]: #compute all the successors and add them to the queue
        new_state = (state[0] | {action}, state[1] - {action})
        #for the distance we consider both cost and the heuristic
        frontier.put((Cost(new_state) + Distance(new_state), new_state))
    
    (_, state) = frontier.get() #new state

print(f"Solve in {counter+1} step by taking sets {state[0]} with cost {Cost(state)}")

In [49]:
#the heuristic proposed by the professor consider the 
#part that we still need to cover and tries to compute the 
#number of sets we will need to cover it all (not considering useless sets)

# Me and Lorenzo tried to use a different approach based on:
# Trying to cover the biggest number of sets while using the least number of tiles
# Since A* tries to minimize f = cost + distance our maximization had to become a minimization problem
# So the heuristic is the number of already covered points negated
# This is obviously admissible, since the heuristic is negative it's always less than the cost
def Distance(state):
    #compute already covered area
    already_covered = reduce(
        np.logical_or,
        [SETS[i] for i in state[0]],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

    return -sum(already_covered)

In [45]:
#we can compare the two with a bigger problem
PROBLEM_SIZE = 25
NUM_SETS = 30
SETS = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

#to compare: run one of the two cell containing Distance (the previous declaration will be deleted)
#then by running this cell you can create new tiles.
#then you can run the A* algorithm and compare the results

Here some results comparing our heuristic (H) and the professor's (PH):
 - First run:
    - H: solved in 5 steps with cost 4
    - PH: solved in 47 steps with cost 4

 - Second run:
    - H: solved in 7 steps with cost 5
    - PH: solved in 567 steps with cost 5
    
 - Third run (here we increased a lot the PROBLEM_SIZE):
    - H: solved in 8 steps with cost 6
    - PH: solved in 2509 steps with cost 6